#  Collections Data to Linked Art - Indianapolis Museum of Art

## Input Data
The [Indianapolis Museum of Art (IMA)]((https://discovernewfields.org/)) has transformed a sample of its collections data to Linked Art JSON-LD:
- available at https://github.com/IMAmuseum/LinkedArt 
- sourced from EMu in XML format (EMu source: Catalogue, Rights, Narratives, and Locations modules)
- [XML files](https://github.com/IMAmuseum/LinkedArt/blob/master/XML)
 - [ObjectsSample XML file](https://github.com/IMAmuseum/LinkedArt/blob/master/XML/ObjectsSample.xml) [raw file](https://raw.githubusercontent.com/IMAmuseum/LinkedArt/master/XML/ObjectsSample.xml)

<div class='alert alert-block alert-info'>An XML file is used as data input for the notebook, and as a miminum the ObjectsSample XML file should be downloaded.</div>

## Attribution

- The notebook's coded transformations are based on the IMA's [XSLT file](https://github.com/IMAmuseum/LinkedArt).
- The Linked Art data model documentation has been sourced from the [Linked Art website](https://linked.art)
- The IMA data has been sourced from the [IMA GitHub repository](https://github.com/IMAmuseum/LinkedArt)

## Transformation Steps

### 1. Import What We Need for Notebook and Define Variable
- Import Python libraries
 - ipywidget
 - IPython
 - xmltodict
 - json
 - requests
 - cromulent

In [76]:
python_libraries = ("ipywidgets", "IPython", "xmltodict", "json","requests","cromulent")

for lib in python_libraries:
    try:
        import lib
    except:
        !pip install lib
        import lib

from ipywidgets import Layout, FileUpload 
from IPython.display import display, IFrame, HTML

from cromulent.model import factory, Actor, Production, BeginningOfExistence, EndOfExistence, TimeSpan, Place
from cromulent.model import InformationObject, Phase, VisualItem 
from cromulent.vocab import Painting, Drawing,Miniature,add_art_setter, PrimaryName, Name, CollectionSet, instances, Sculpture 
from cromulent.vocab import aat_culture_mapping, AccessionNumber, Height, Width, SupportPart, Gallery, MuseumPlace 
from cromulent.vocab import BottomPart, Description, RightsStatement, MuseumOrg, Purchase
from cromulent.vocab import Furniture, Mosaic, Photograph, Coin, Vessel, Graphic, Enamel, Embroidery, PhotographPrint
from cromulent.vocab import PhotographAlbum, PhotographBook, PhotographColor, PhotographBW, Negative, Map, Clothing, Furniture
from cromulent.vocab import Sample, Architecture, Armor, Book, DecArts, Implement, Jewelry, Manuscript, SiteInstallation, Text, Print
from cromulent.vocab import TimeBasedMedia, Page, Folio, Folder, Box, Envelope, Binder, Case, FlatfileCabinet
from cromulent.vocab import HumanMadeObject,Tapestry


In [73]:


objTypes = {
"Painting": Painting(),
"Sculpture": Sculpture(),
"Drawing": Drawing(),
"Miniature": Miniature(),
"Tapestry": Tapestry(),
"Furniture": Furniture(),
"Furnishings": DecArts(),
"Mosaic": Mosaic(),
"Photograph": Photograph(),
"Coin": Coin(),
"Vessel": Vessel(),
"Graphic": Graphic(),
"Enamel": Enamel(),
"Embroidery": Embroidery(),
"PhotographPrint": PhotographPrint(),
"PhotographAlbum": PhotographAlbum(),
"PhotographBook": PhotographBook(),
"PhotographColor": PhotographColor(),
"PhotographBW": PhotographBW(),
"Negative": Negative(),
"Map": Map(),
"Clothing": Clothing(),
"Sample": Sample(),
"Architecture": Architecture(),
"Armor": Armor(),
"Book": Book(),
"DecArts": DecArts(),
"Implement": Implement(),
"Jewelry": Jewelry(),
"Manuscript": Manuscript(),
"SiteInstallation": SiteInstallation(),
"Text": Text(),
"Print": Print(),
"TimeBasedMedia": TimeBasedMedia(),
"Page": Page(),
"Folio": Folio(),
"Folder": Folder(),
"Box": Box(),
"Envelope": Envelope(),
"Binder": Binder(),
"Case": Case(),
"FlatfileCabinet": FlatfileCabinet()
}



### 2. Upload XML File
- Choose a file on your local system to upload and transform to Linked Art
- The IMA files are available to download from: https://github.com/IMAmuseum/LinkedArt/tree/master/XML
- Select a file by clicking on the "Select XML file" button

In [74]:
upload = FileUpload(accept='.xml', multiple=False, description='Select XML file')

In [75]:
display(HTML("<div class='alert alert-block alert-info'>Please select a file to transform</div>"))
display(upload)

FileUpload(value={}, accept='.xml', description='Select XML file')

In [60]:
obj = False
# get content from uploaded file 
for uploaded_filename in upload.value:
    content = upload.value[uploaded_filename]['content']
    obj = xmltodict.parse(content) 
    
if obj == False:
    display(HTML("<div class='alert alert-block alert-danger'>Please select a file to transform</div>"))
else:
    display(HTML("<div class='alert alert-block alert-success'>File uploaded</div>"))

### 3. Select Catalogue Item to Transform
The next step will transform the contents of the uploaded XML file into a dropdown list of artworks.

In [61]:
allObjects = obj["table"]["tuple"]

#object dropdown options
objOptions = []
objOptions.append(("Please select an object",''))

for obj in allObjects:
    title = irn = ""
    # define properties variables  
    for prop in obj["atom"]:      
        propName = prop["@name"]     
        if "#text" in prop:
            if propName == "irn":
                irn = prop["#text"]  
            if propName == "TitMainTitle":
                title = prop["#text"]
            
    objOptions.append((irn + ' : ' + title,irn))
    
selectObject = widgets.Dropdown(options=objOptions, description='Select Object')

In [63]:
display(HTML("<div class='alert alert-block alert-info'>Select an artwork from the dropdown list to transform the catalogue data to Linked Art JSON-LD</div>"))

display(selectObject)

Dropdown(description='Select Object', index=2, options=(('Please select an object', ''), ('1032 : long-neck va…

In [64]:
selectedIRN = selectObject.value

if selectedIRN == "":
    display(HTML("<div class='alert alert-block alert-danger'>Please select an artwork to transform</div>"))
else:
    display(HTML("<div class='alert alert-block alert-success'>Artwork selected:" + selectObject.options[selectObject.index][0] + "</div"))  


### Artwork as Python dictionary
The following Python dictionary is for the selected artwork, and will be used to transform the collection data for the artwork to JSON-LD:

In [65]:
selectedObject = {}
for obj in allObjects:
    for prop in obj["atom"]:         
        if prop["@name"] == "irn":
            irn = prop["#text"]
            if irn == selectedIRN:
                selectedObject = obj
                break
               
obj = selectedObject

print(json.dumps(obj,indent=2))


{
  "atom": [
    {
      "@name": "irn",
      "@type": "text",
      "@size": "short",
      "#text": "1065"
    },
    {
      "@name": "AdmPublishWebNoPassword",
      "@type": "text",
      "@size": "short",
      "#text": "Yes"
    },
    {
      "@name": "TitAccessionNo",
      "@type": "text",
      "@size": "short",
      "#text": "14.55"
    },
    {
      "@name": "TitPreviousAccessionNo",
      "@type": "text",
      "@size": "short",
      "#text": "No TR Number"
    },
    {
      "@name": "TitObjectStatus",
      "@type": "text",
      "@size": "short",
      "#text": "Accessioned"
    },
    {
      "@name": "TitAccessionDate",
      "@type": "text",
      "@size": "short",
      "#text": "1914-02-19"
    },
    {
      "@name": "TitMainTitle",
      "@type": "text",
      "@size": "short",
      "#text": "The Bachelor Boudoir (Corner of the Studio)"
    },
    {
      "@name": "TitSeriesTitle",
      "@type": "text",
      "@size": "short"
    },
    {
      "@name": "

## 4. Build the Linked Art JSON-LD file <a id="build"/>

The following steps will transform the catalogue data for the selected artwork to Linked Art JSON-LD. The transformation will be divided into sections, using different parts of the Linked Art data model.


### Mapping of Input Fields

In [18]:
docProp = {
    "irn":"id",
    "TitAccessionNo":"accession_number",
    "TitAccessionDate": "accession_date",
    "TitObjectType" : "classification",
    "TitObjectStatus" : "current_status",
    "TitMainTitle": "title",
    "TitTitleNotes": "notes",
    "CreDateCreated":"date_created",
    "CreEarliestDate": "date_created_earliest",
    "CreLatestDate": "date_created_latest",
    "CreCreationPeriod":"created_period",
    "CreCreationDynasty":"created_dynasty",
    "CrePrimaryInscriptions":"inscriptions",
    
    "CreCreationNotes": "created_notes",
    "PhyMediumAndSupport": "physical_medium",
    "PhyStyle": "physical_style",
    "PhyTechnique": "physical_technique",
    "PhyDescription": "physical_description",
    "PhyConvertedDims": "physical_dimensions",
    "PhyCollectionArea" : "PhyCollectionArea" ,
    "CreProvenance": "created_provenance" 
    
}


mapp =  {
    "id":"id",
    "accession_number":"accession_number",
    "accession_date": "accession_date",
    "classification" : "classification",
    "title": "title",
    "alt_title": "alt_title",
    "notes": "notes",
    "date_created":"date_created",
    "date_created_earliest": "date_created_earliest",
    "date_created_latest": "date_created_latest",
    "created_period":"created_period",
    "created_dynasty":"created_dynasty",
    "created_inscriptions":"inscriptions",
    "created_notes": "created_notes",
    "creator":"creator",
    "physical_medium": "physical_medium",
    "physical_style": "physical_style",
    "physical_technique": "physical_technique",
    "physical_description": "physical_description",
    "physical_dimensions": "physical_dimensions",
    "created_provenance": "created_provenance" ,
    "credit_line": "credit_line",
    "collection" : "PhyCollectionArea",
    "current_status" : "current_status"
}


def createObjProp(obj,docProp):
    objProp = {"creator":[]}

    for prop in obj["atom"]: 
        propName = prop["@name"]
        propValue = ""
        if "#text" in prop:
            propValue = prop["#text"]
        if propName in list(docProp.keys()):
            propId = docProp[propName]
            objProp[propId] = propValue
 

    # alternative titles

    for table in obj["table"]:
        if table["@name"] == "AltTitles": 
            alt_title = table["tuple"]["atom"]["#text"]
            objProp["alt_title"] = alt_title
        

        if table["@name"] == "Creator1": 
            crerole = creid = crename = ""
            if "atom" in table["tuple"]:
                for atom in table["tuple"]["atom"]:
                    if atom["@name"] == "CreRole":
                        crerole = atom["#text"]
                    if atom["@name"] == "irn":
                        creid = atom["#text"]
                    if atom["@name"] == "SummaryData":
                        crename = atom["#text"]
            creator = {"id":creid,"name":crename,"role":crerole}
            objProp["creator"].append(creator)
    
    return objProp


objProp = createObjProp(obj,docProp)      
print(json.dumps(objProp,indent=2))

{
  "creator": [
    {
      "id": "613279",
      "name": "Lichtenstein, Roy",
      "role": "Artist"
    }
  ],
  "id": "82696",
  "accession_number": "2013.443A-E.4",
  "current_status": "Accessioned",
  "accession_date": "2013-12-12",
  "title": "Five Brushstrokes",
  "notes": "A) Element A / Black and Yellow Drip, B) Element B / Red and White, C) Element C / Black & White, E.1) Element E #1 / Bottom, Black & White, E.2) Element E #2 / Red & White, E.3) Element E #3 / Black &  Yellow, E.4) Element E #4 / Top, Black & White",
  "classification": "Visual Works: Sculpture",
  "date_created": "fabricated 2012",
  "date_created_earliest": "2012",
  "date_created_latest": "2012",
  "created_period": "",
  "created_dynasty": "",
  "inscriptions": "rf Lichtenstein 83-84 AP <copyright> The Roy Lichtenstein Foundation, 2012",
  "created_notes": "(designed 1983-1984, fabricated 2012)\nThe sculpture was created posthumously from the aritist's specificiations, and originally commissioned by by 

[Linked Art Data Model documentation](https://linked.art/model/base/#core-properties)

There are a few core properties that every resource should have for it to be a useful part of the world of Linked Open Data:

- `@context`
- `id`
- `type`
- `_label`
    
---

#### IMA Data Mapping

The `id` is a URL and has been created from the `irn` value together with a URL prefix: https://data.discovernewfields.org/

The `_label` is a human readable label, intended for developers and other people reading the data. The value is taken from the `TitMainTitle` property.


In [19]:
#  baseURI for JSON-LD document
baseURI = "https://data.discovernewfields.org/"

factory.base_url = baseURI
factory.default_lang = "en"

## Classification

https://linked.art/model/base/#types-and-classifications

CIDOC-CRM is a framework that must be extended via additional vocabularies and ontologies to be useful. The provided mechanism for doing this is the classified_as property, which refers to a term from a controlled vocabulary. This is in contrast to the `type` property, which is used for CIDOC-CRM defined classes, and a few extensions as needed. 

The `classified_as` property is thus a way to be more specific about the sort of entity, while maintaining the core information as the class using type. Controlled vocabulary entries should not be used with `type`, nor classes used with `classified_as`.

While any external vocabulary of terms can be used, the Getty's Art and Architecture Thesaurus is used whenever possible for consistency and that it is already widespread in the museum domain. The set of terms that have been identified as useful are listed in the community best-practices for recommendations, and within the documentation of the model when a particular choice is essential for interoperability.


## Identifiers <a id="id"/>

[Linked Art Data Model documentation](https://linked.art/model/base/#identifiers)

Many resources of interest are also given external identifiers, such as accession numbers for objects, ORCIDs for people or groups, lot numbers for auctions, and so forth. Identifiers are represented in a very similar way to names, but instead use the Identifier class. Identifiers will normally have a classification determining which sort of identifier it is, to distinguish between internal repository system assigned numbers from museum assigned accession numbers, for example.

As Identifiers and Names use the same `identified_by` property, the JSON will frequently have mixed classes in the array. Unlike `Names`, `Identifiers` are not part of human language and thus cannot have translations or a language associated with them.

In [20]:
id = objProp[mapp.get("id")]

object_uri = baseURI + id

### Accession Number

In [21]:
from cromulent.vocab import AccessionNumber

def objAccession(objProp, mapp,object_uri):
    accession = None
    accession_number = objProp[mapp.get("accession_number")]
    if accession_number != "":
        accession = AccessionNumber(accession_number,value=accession_number)
    return accession

In [22]:
accession = objAccession(objProp,mapp,object_uri)
print(factory.toString(accession, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/Identifier/2013.443A-E.4",
  "type": "Identifier",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300312355",
      "type": "Type",
      "_label": "Accession Number"
    }
  ],
  "content": "2013.443A-E.4"
}


### Local Number

In [23]:
from cromulent.vocab import LocalNumber

def objLocalnumber(objProp,mapp,object_uri):
    localnumber = None
    id = objProp[mapp.get("id")]
    if id != "":
        localnumber = LocalNumber(id,value=id)
    return localnumber

In [24]:
localnumber = objLocalnumber(objProp,mapp,object_uri)
print(factory.toString(localnumber, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/Identifier/82696",
  "type": "Identifier",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300404621",
      "type": "Type",
      "_label": "Owner-Assigned Number"
    }
  ],
  "content": "82696"
}


## Names 

https://linked.art/model/base/#names

As the `_label` property is intended as internal documentation for the data, it is strongly recommended that every resource that should be rendered to an end user also have at least one specific name. The name could be for an object, a person, a group, an event or anything else. This pattern uses the `identified_by` property, with a `Name` resource. The value of the name is given in the content property of the `Name`.

It is somewhat unintuitive to think of a name as identifying the resource it is associated with, as names are typically not unique. However, as the name itself is uniquely identified rather than just an anonymous string, they are no longer a shared label and instead the particular instance of a name is uniquely associated with the resource. With this formulation, the name instance does uniquely identify the resource.

If there is more than one name given, then there should be one that is `classified_as` the primary name for use. This is done by adding the `Primary Name` (aat:300404670) term to it. There should be exactly one primary title given per language.

Names are also part of human communication, and can have the Linguistic features of the model associated with them, such as having a particular language, or having translations.



### Primary Name

In [25]:
from cromulent.vocab import PrimaryName

def objPrimaryname(objProp,mapp,object_uri):
    primaryname = None
    title = objProp[mapp["title"]]
    id = objProp[mapp["id"]]
    primaryname = PrimaryName( object_uri + id + "/primary-name",
                                value=title)
    return primaryname

In [26]:
primaryname = objPrimaryname(objProp,mapp,object_uri) 
print(factory.toString(primaryname, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/8269682696/primary-name",
  "type": "Name",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300404670",
      "type": "Type",
      "_label": "Primary Name"
    }
  ],
  "content": "Five Brushstrokes"
}


### Alternate Name

In [27]:
from cromulent.vocab import AlternateName

def objAlternatename(objProp,mapp,object_uri):
    alternateName = None
    if mapp["alt_title"] in objProp:
        alt_title = objProp[mapp["alt_title"]]
        alternatename = AlternateName(object_uri + id + "/alternate-name",value=alt_title)
    return alternateName

In [28]:
alternatename = objAlternatename(objProp,mapp,object_uri) 

if alternatename is not None:
    print(factory.toString(alternatename, compact=False))

## Home Page

https://linked.art/model/digital/#home-page

A very common scenario is that there is a web page about the object, perhaps managed by a collections management system. For humans, this page is much more useful than the data intended for machines. It can be referenced with the `subject_of` property, and points to a `DigitalObject` which is `classified_as` a web page, or `aat:300264578`. As with digital images, the home page can have a format of "text/html" and other properties.

In [29]:
from cromulent.vocab import LinguisticObject, Type

def objHomepage(objProp,mapp,object_uri):

    homepage = None
    id = objProp[mapp["id"]]

    homepageId = "http://collection.imamuseum.org/artwork/" + id

    homepage = LinguisticObject(homepageId, label="Homepage for the Object")
    homepage.classified_as = Type("http://vocab/getty.edu/aat/300264578", label="Web pages (documents)")
    homepage.classified_as = Type("http://vocab.getty.edu/aat/300266277", label="home pages")
    homepage.format = "text/html"
    
    return homepage

In [30]:
homepage = objHomepage(objProp,mapp,object_uri) 

if homepage is not None:
    print(factory.toString(homepage, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://collection.imamuseum.org/artwork/82696",
  "type": "LinguisticObject",
  "_label": "Homepage for the Object",
  "classified_as": [
    {
      "id": "http://vocab/getty.edu/aat/300264578",
      "type": "Type",
      "_label": "Web pages (documents)"
    },
    {
      "id": "http://vocab.getty.edu/aat/300266277",
      "type": "Type",
      "_label": "home pages"
    }
  ],
  "format": "text/html"
}


## Statements about a Resource - Linguistic Objects <a id="ling"/>

[Linked Art Data Model Documentation](https://linked.art/model/base/#statements-about-a-resource)
    
In many cases, current data does not support the level of specificity that the full ontology allows, or the information is simply best expressed in human-readable form. For example, instead of a completely modeled set of parts with materials, many museum collection management systems allow only a single human-readable string for the "medium" or "materials statement". The same is true in many other situations, including rights or allowable usage statements, dimensions, edition statements and so forth. Any time that there is a description of the resource, with or without qualification as to the type of description, then this pattern can be used to record the descriptive text.

The pattern makes use of the `LinguisticObject` class that is used to identify a particular piece of textual content. These Linguistic Objects are then refered to by any other resource. They maintain the statement's text in the content property, and the language of the statement (if known) in the language property.

Use cases for this pattern include:

- General description of the resource
- Materials statement for an object
- Attribution statement for an image
- Biography for a person
- Dimensions statement for a part of an object    


## Provenance

In [31]:
from cromulent.vocab import LinguisticObject, Type

def objProvenance(objProp,mapp,object_uri):

    prov = None
    propProv = mapp["created_provenance"]
    if propProv in objProp:
        provenance = objProp[propProv]
        if provenance != "":
            prov = LinguisticObject(object_uri + "/provenance-statement", 
                            value=provenance,
                            label="Provenance Statement about the Object"
                           )
            prov.classified_as = Type("http://vocab.getty.edu/aat/300055863", label="provenance (history of ownership)")
            prov.classified_as = Type("http://vocab.getty.edu/aat/300418049", label="brief texts")
    return prov

In [32]:
prov = objProvenance(objProp,mapp,object_uri)

if prov is not None:
    print(factory.toString(prov, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/82696/provenance-statement",
  "type": "LinguisticObject",
  "_label": "Provenance Statement about the Object",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300055863",
      "type": "Type",
      "_label": "provenance (history of ownership)"
    },
    {
      "id": "http://vocab.getty.edu/aat/300418049",
      "type": "Type",
      "_label": "brief texts"
    }
  ],
  "content": "(Roy Lichtenstein Foundation, New York, New York); purchased by the Indianapolis Museum of Art in 2013."
}


## Credit Line

In [33]:
from cromulent.vocab import LinguisticObject

def objCredit(objProp,mapp,object_uri):
    credit = None
    propCredit = mapp["credit_line"]
    
    if propCredit in objProp:
        credit_line = objProp[propCredit]
        if credit_line != "":
            credit = LinguisticObject(baseURI + id + "/credit-line", 
                            value=credit_line,
                            label="Credit Line for the Object"
                           )
            credit.classified_as = Type("http://vocab.getty.edu/aat/300026687", label="acknowledgements")
            credit.classified_as = Type("http://vocab.getty.edu/aat/300418049", label="brief texts")
    return credit

In [34]:
credit = objCredit(objProp,mapp,object_uri)

if credit is not None:
    print(factory.toString(credit, compact=False))

## Production <a id="prod"/>

[Linked Art Data Model Documentation](https://linked.art/model/object/production/)

The first activity in an object's lifecycle is its creation, or `Production`. The relationship to the object that was produced by the activity (`produced`) is added to the general activity model, along with the time, location and actors. This follows the base pattern for activities.


In [35]:
from cromulent.vocab import Production, TimeSpan, Actor

def objProduction(objProp,mapp,object_uri):

    prod = None

    date_created = mapp["date_created"]
    created_earliest = mapp["date_created_earliest"]
    created_latest = mapp["date_created_latest"]

    if date_created in objProp:
        prod = Production(object_uri + "/production", label="Production of the Object")    
    
        labelTimespan = "date unknown"
        if objProp[date_created] != "":
            labelTimespan = objProp[date_created]
   
        timespan = TimeSpan(object_uri + "/production/timespan", label=labelTimespan)
        
        if created_earliest in objProp:
            timespan.begin_of_the_begin = objProp[created_earliest]
        if created_latest in objProp:
            timespan.end_of_the_end = objProp[created_latest]
    
        prod.timespan = timespan
    
        for creator in objProp["creator"]:
            if "id" in creator:
                actor = Actor(creator["id"], label=creator["name"])
                prod.carried_out_by = actor
    return prod

In [36]:
prod = objProduction(objProp,mapp,object_uri)

if prod is not None:
    print(factory.toString(prod, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/82696/production",
  "type": "Production",
  "_label": "Production of the Object",
  "timespan": {
    "id": "https://data.discovernewfields.org/82696/production/timespan",
    "type": "TimeSpan",
    "_label": "fabricated 2012",
    "begin_of_the_begin": "2012",
    "end_of_the_end": "2012"
  },
  "carried_out_by": [
    {
      "id": "https://data.discovernewfields.org/Actor/613279",
      "type": "Actor",
      "_label": "Lichtenstein, Roy"
    }
  ]
}


## Current Owner and Acquisition <a id="owner"/>

[Linked Art Data Model Documentation](https://linked.art/model/provenance/acquisition/#object-acquisition)

Acquisitions are used to describe the transfer of ownership of an object from one owner to the next. The first owner is typically the artist, who would then transfer it to the second owner, to the third owner and so on. The ownership chain can be expressed by repeating this same pattern with the buyer from one acquisition being the seller in the subsequent one. If the previous owner (e.g. the seller if there is a value exchange) or the subsequent owner (e.g. the buyer) is not known for a particular acquisition, then the reference can be left out from the description.

The acquistion is not necessarily a purchase, it could be a gift, an inheritance or any other method of gaining the right of ownership of an object.

The model encodes this information with an Acquisition part of the overall Provenance Event. The acquisition is the transfer of the right of ownership of an object (referenced in transferred_title_of) from the seller (in transferred_title_from) to the buyer (in transferred_title_to).

Each object has its own Acquisition as part of the provenance event, so if a collector buys three paintings from a dealer, then there would be a single Provenance Event with three Acquisitions, all of which transfer the title of a single painting from the dealer to the collector.


In [37]:
from cromulent.vocab import TimeSpan, Actor, Group, Acquisition, Place

def checkCurrentOwner(current_status):
    currentowner = False
    if current_status != "":
        checkObjStatus = ('Accessioned','Partial Accession')
        for status in checkObjStatus:
            if status == current_status:
                currentowner = True
            if 'IMA-Owned' in current_status:
                currentowner = True
    return currentowner


def objCurrentowner(objProp,mapp,object_uri):
    current_owner = None
    
    current_status = objProp[mapp["current_status"]]
    currentowner = checkCurrentOwner(current_status)


    if currentowner == True:
        current_owner = Group("http://vocab.getty.edu/ulan/500300517", 
                          label="Indianapolis Museum of Art at Newfields"
                         )
        current_owner.classified_as = Type("http://vocab.getty.edu/aat/300312281",
                                      label="museums (institutions)")
    
        accession_date = objProp[mapp["accession_date"]]

         
        acquisition = objAcquisition(object_uri,objProp,mapp)
        if acquisition is not None:
            current_owner.acquired_title_through = acquisition 
    return current_owner 


def objAcquisition(object_uri,objProp,mapp):
    acquisition = None
    accession_date = objProp[mapp["accession_date"]]
    if accession_date != "": 
        acquisition = Acquisition(object_uri + "/IMA-acquisition", label = "Acquisition of the Object")
        acquisition.classified_as = Type("http://vocab.getty.edu/aat/300157782",label="acquisition (collections management)")
        acquisition.took_place_at = Place("http://vocab.getty.edu/tgn/7012924", label="Indianapolis, Indiana") 
        acquisition.timespan = objAcquisitionTimespan(object_uri,accession_date)
    return acquisition

def objAcquisitionTimespan(object_uri,accession_date):
    timespan = None
    end = begin = ""
    if len(accession_date) == 4:
        begin = accession_date + "-01-01T00:00:00.000Z"
        end = accession_date + "-12-31T00:00:00.000Z"
        
    elif len(accession_date) == 8:
        begin = accession_date + "01T00:00:00.000Z"
        end = accession_date 
        if '-02-' in accession_date:
            end = end + "28"
        if ('-01-','-03-','-05-','-07-','-08-','-09-','-10-','-12-') in accession_date:
            end = end + "31"
        if ('-04-','-06-','-09-','-11-'):
            end = end + "30"      
        end = end + "T00:00:00"
         
    elif len(accession_date) == 10:
        begin = accession_date + "T00:00:00.000Z"
        end = accession_date + "T00:00:00.000Z"    
    else:
        begin = end = ""
        
    timespan = TimeSpan(object_uri + "/IMA-acquisition/timespan", label=accession_date)
        
    if begin != "":
        timespan.begin_of_the_begin = begin
    if end != "":
        timespan.end_of_the_end = end
    return timespan
    

In [38]:
current_owner = objCurrentowner(objProp,mapp,object_uri)

if current_owner is not None:
    print(factory.toString(current_owner, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "http://vocab.getty.edu/ulan/500300517",
  "type": "Group",
  "_label": "Indianapolis Museum of Art at Newfields",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300312281",
      "type": "Type",
      "_label": "museums (institutions)"
    }
  ],
  "acquired_title_through": [
    {
      "id": "https://data.discovernewfields.org/82696/IMA-acquisition",
      "type": "Acquisition",
      "_label": "Acquisition of the Object",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300157782",
          "type": "Type",
          "_label": "acquisition (collections management)"
        }
      ],
      "timespan": {
        "id": "https://data.discovernewfields.org/82696/IMA-acquisition/timespan",
        "type": "TimeSpan",
        "_label": "2013-12-12",
        "begin_of_the_begin": "2013-12-12T00:00:00.000Z",
        "end_of_the_end": "2013-12-12T00:00:00.000Z"
      },
    

## Custody

[Linked Art Data Model documentation](https://linked.art/model/provenance/custody/#institutional-ownership-departmental-custody)

Objects are owned by legal entities, such as museum organizations or individual people. However there may be more information about which department is responsible within a museum for the curation of the object. This is the division between acquisitions (the legal ownership of the object) and custody (the responsibility for looking after the object). If the department is known, then it should be either part of the Provenance Event in which the object is acquired, or a separate provenance event if the object was not accessioned by a department and later came under their care, or was transferred between departments. In these latter cases, the ownership does not change, only the custody of the object.

The department becomes the `current_keeper` of the object, whereas the institution is the `current_owner`.


In [39]:
from cromulent.vocab import Group, Type


def objCustody(objProp,mapp,object_uri):
    
    custody = None
    current_status = objProp[mapp["current_status"]]

    currentowner = checkCurrentOwner(current_status)
    
    if currentowner == False:
        custody = Group(label="Indianapolis Museum of Art at Newfields")
        custody.classified_as = Type("http://vocab.getty.edu/aat/300312281", 
                                     label="museums (institutions)")
    return custody

In [40]:
custody = objCustody(objProp,mapp,object_uri)
if custody is not None:
    print(factory.toString(custody, compact=False))

## Membership of Collections and Sets <a id="member"/>

[Linked Art Data Model Documentation](https://linked.art/model/collection/)

There are many use cases for grouping resources together, often of the same class but sometimes of varying types. These use cases are exemplified in the sections below, and range from the set of objects in an auction lot, to dealer inventories and museum collections, exhibitions, a set of related concepts, or the set of people that share a common feature such as gender or nationality.

In order to cover all of the use cases with a consistent pattern, we introduce a new `Set` class from outside of CIDOC-CRM. This avoids issues with sets of resources with different types, and the semantics of the identity of objects and collections. If an equivalent class is added into the core CIDOC-CRM ontology in the future, a new major version of the specification will change to using it.

### Sets 

Core Features

Sets are conceptual groupings, rather than physical ones. The set of objects in a virtual exhibition or simply the set of a person's favorite objects never change their physical state by being part of the Set or not. They are, thus, created by a `Creation`, not by a `Production`.

Like any core resource, Set must have an id and type, are likely to have additional classifications, and can have `Identifiers` and `Names`. They can have statements made about them, and have member resources. These member resources are included via the `member` property rather than `part`, or via `member_of` from the included resource to the `Set`.


In [41]:
from cromulent.vocab import Set

def objCollection(objProp,mapp,object_uri):
    coll = None
    collection = mapp["collection"]

    if collection in objProp:
        collection = objProp[collection]
        coll = Set(object_uri +"/collection/" + collection, 
                   label="Collection of IMA at Newfields' " + collection.split("-",1)[1]  + " Department")
        coll.classified_as = Type("http://vocab.getty.edu/aat/300025976", 
                              label="collections (object groupings)")
    return coll

In [42]:
coll = objCollection(objProp,mapp,object_uri)
if coll is not None:
    print(factory.toString(coll, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/82696/collection/907-Contemporary",
  "type": "Set",
  "_label": "Collection of IMA at Newfields' Contemporary Department",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300025976",
      "type": "Type",
      "_label": "collections (object groupings)"
    }
  ]
}


## 5. Compile Linked Art JSON-LD Representation

Bring the separate descriptions together in a single JSON-LD document

In [43]:
def createObjDesc(objProp,mapp,objTypes,object_uri):
    objLA = None
    objLA = HumanMadeObject() # linked art object

    for otype in objTypes:
        if otype in objProp[mapp.get("classification")]:
            objLA = objTypes[otype]        
            break
            
    objLA.id = object_uri
    objLA._label =  objProp[mapp.get("title")]

    
    # IDENTIFIED_BY 
    accession = objAccession(objProp,mapp,object_uri)
    localnumber = objLocalnumber(objProp,mapp,object_uri)
    primaryname = objPrimaryname(objProp,mapp,object_uri)
    
    listIds = (accession,localnumber,primaryname)
    identified_by = False
    for id in listIds:
        if id is not None:
            identified_by = True
            break
    if identified_by == True:
        objLA.identified_by = []
        
        for id in listIds:
            if id is not None:
                objLA.identified_by.append(id)
    
    # REFERRED_TO_BY
    prov = objProvenance(objProp,mapp,object_uri)
    credit = objCredit(objProp,mapp,object_uri)
    referred_to_by = False
    if prov is not None or credit is not None:
        referred_to_by = True
    if referred_to_by == True:
        objLA.referred_to_by = []
    if prov is not None:
        objLA.referred_to_by.append(prov) # provenance 
    if credit is not None:
        objLA.referred_to_by.append(credit) # credit line
    
    # SUBJECT_OF    
    homepage = objHomepage(objProp,mapp,object_uri)
    if homepage is not None:
        objLA.subject_of = homepage # home page
        
    # PRODUCED_BY
    prod = objProduction(objProp,mapp,object_uri)
    objLA.produced_by = None
    if prod is not None:
        objLA.produced_by = prod # production

    # MEMBER_OF
    coll = objCollection(objProp,mapp,object_uri)
    if coll is not None:
        objLA.member_of = coll # collection

    # CURRENT_KEEPER
    custody = objCustody(objProp,mapp,object_uri)
    if custody is not None:
        objLA.current_owner = custody
    
    # CURRENT_OWNER
    current_owner = objCurrentowner(objProp,mapp,object_uri)
    if current_owner is not None:
        objLA.current_owner = current_owner
          
    return objLA

In [44]:
objProp = createObjProp(obj,docProp)

ObjLA = None
objLA = createObjDesc(objProp,mapp,objTypes,object_uri)

## 7. View the final Linked Art JSON-LD

The final Linked Art JSON-LD representation of the object is now available:

In [45]:
print(factory.toString(objLA, compact=False))
f = open("./data/ima/output/json/object_linkedart.json", "w")
f.write(factory.toString(objLA, compact=False))
f.close() 

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://data.discovernewfields.org/82696",
  "type": "HumanMadeObject",
  "_label": "Five Brushstrokes",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300047090",
      "type": "Type",
      "_label": "Sculpture",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300435443",
          "type": "Type",
          "_label": "Type of Work"
        }
      ]
    }
  ],
  "identified_by": [
    {
      "id": "https://data.discovernewfields.org/Identifier/2013.443A-E.4",
      "type": "Identifier",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300312355",
          "type": "Type",
          "_label": "Accession Number"
        }
      ],
      "content": "2013.443A-E.4"
    },
    {
      "id": "https://data.discovernewfields.org/Identifier/82696",
      "type": "Identifier",
      "classified_as": [
        {
          "id": "http://vocab.gett

## 8. View the Linked Art JSON-LD file

The final JSON-LD file for the selected object is available at:
- [Selected Object as Linked Art](./data/ima/output/json/object_linkedart.json)

View the JSON-LD in the [JSON-LD Playground](https://json-ld.org/playground)

## 9. Transform All Artworks to Linked Art JSON-LD

In [46]:
for obj in allObjects:
    objProp = createObjProp(obj,docProp)
    
    id = objProp[mapp.get("id")]

    object_uri = baseURI + id
    objLA = createObjDesc(objProp,mapp,objTypes,object_uri)
    text_file = open("./data/ima/output/json/all/" + objProp["id"] + ".json", "wt")
    n = text_file.write(factory.toString(objLA, compact=False))
    text_file.close()

View directory listing of Linked Art JSON-LD files

In [47]:
import os
from IPython.core.display import display, HTML


def fn():       # 1.Get file names from directory
    file_list=os.listdir(r"./data/ima/output/json/all/")
   
    for file in file_list:
        display(HTML("<a target='_new' href='./data/ima/output/json/all/" + file +"'>" + file + "</a>"))
    
fn()

## 10. Next Steps

Visit the [Linked Art community website](https://linked.art) for further information on the data model and community activities.

In [71]:
def write2file(filename, obj):
    # write to file
    f = open("./data/ima/output/json/" + filename + ".json", "w")
    f.write(json.dumps(obj, indent=2))
    f.close() 
     